Import modules and prep training data

In [1]:
from Pfeature.pfeature import aac_wp
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix
import pandas as pd
import os
import lazypredict
from lazypredict.Supervised import LazyClassifier

# prep positive data and label
aac_wp('/home/jovyan/work/Short_AMPs_Predictor/train_set/po_cd_hit.txt', '/home/jovyan/work/Short_AMPs_Predictor/po_aac.csv')
po_df = pd.read_csv('po_aac.csv')
po_df['y'] = 1
po_df

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/work/Short_AMPs_Predictor/train_set/po_cd_hit.txt'

In [ ]:
# prep negative data and label
aac_wp('train_set/ne_cd_hit.txt', 'ne_aac.csv')
ne_df = pd.read_csv('ne_aac.csv')
ne_df['y'] = 0
ne_df

In [ ]:
os.remove('po_aac.csv')
os.remove('ne_aac.csv')

Combine po and ne data and shuffle

In [ ]:
X_train = pd.concat([po_df, ne_df], axis=0)
# shuffle data
X_train = X_train.sample(frac=1).reset_index(drop=True)

Seperate X and y data from training

In [ ]:
y_train = X_train['y']
X_train.drop(columns='y', inplace=True)
X_train

In [ ]:
y_train

Find algorithm with best accuracy

In [ ]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=matthews_corrcoef)
models_train, predictions_train = clf.fit(X_train, X_train, y_train, y_train)
models_train

Setup testing data to try and apply to model

In [ ]:
aac_wp('test_set/po_cd_hit.txt', 'po_cd_hit.csv')
aac_wp('test_set/ne_cd_hit.txt', 'ne_cd_hit.csv')
test_po_df = pd.read_csv('po_cd_hit.csv')
test_ne_df = pd.read_csv('ne_cd_hit.csv')
y_test = [1 for i in range(test_po_df.index.stop)] + [0 for i in range(test_ne_df.index.stop)]
os.remove('po_cd_hit.csv')
os.remove('ne_cd_hit.csv')
X_test = pd.concat([test_po_df, test_ne_df], axis=0)
X_test

In [ ]:
# models_test, predictions_test = clf.fit(X_train, X_test, y_train, y_test)
# models_test

Random Forest is the best so setup for that

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=500)
rf.fit(X_train, y_train)

Apply model

In [ ]:
# _predict = []

# for i in range(len(X_test)):
#     prediction = rf.predict(X_test.iloc[i:i+1, :])
#     _predict.append(prediction)

y_test_pred = rf.predict(X_test)
y_test_pred

Some scores

In [ ]:
accuracy_score(y_test, y_test_pred), matthews_corrcoef(y_test, y_test_pred), confusion_matrix(y_test, y_test_pred)

Remove features that don't mean anything using variance

In [ ]:
# all thresholds for aac will ussually be way greater than 0.1
threshold = VarianceThreshold(threshold=0.1)
threshold.fit_transform(X_train)
X2 = X_train.loc[:, threshold.get_support()]
X2

Show features by importance

In [ ]:
final_df = pd.DataFrame(data={'Features': X2.columns, 'Gini' : rf.feature_importances_})
final_df.sort_values(by='Gini', ascending=False, inplace=True)
final_df.reset_index(drop=True, inplace=True)
final_df

Also show in graphical representation

In [ ]:
# Combine feature names and Gini values into a Dataframe
# Plot of feature importance
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(5, 10))
sns.set_theme(style="whitegrid")
ax = sns.barplot(x = 'Gini', y = 'Features', data = final_df)
plt.xlabel("Feature Importance")